## 0.환경준비

* 라이브러리 불러오기

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 타이타닉 데이터
titanic = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/titanic.1.csv')
titanic.head()

In [ ]:
# 코스피 지수
stock = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/KOSPI.csv')
stock['Date'] = pd.to_datetime(stock['Date'])
stock['Year'] = stock.Date.dt.year
stock['Month'] = stock.Date.dt.month
stock['Weekday'] = stock.Date.dt.weekday
stock.head()

## 1.교차표(pd.crosstab)

범주 vs 범주 를 비교하고 분석하기 위해서는 먼저 **교차표**를 만들어야 합니다.

* pd.crosstab(행, 열)

In [ ]:
# 두 범주별 빈도수를 교차표로 만들어 봅시다.
pd.crosstab(titanic['Survived'], titanic['Sex'])

In [ ]:
pd.crosstab(titanic['Survived'], titanic['Embarked'])

* pd.crosstab(행, 열, normalize =   )

<img src='https://github.com/DA4BAM/image/blob/main/crosstab.png?raw=true' width=700/>


* x를 기준으로 y의 비율을 살펴본다.

In [ ]:
# pd.crosstab( y, x, normalize = 'columns') <== 권장!
pd.crosstab(titanic['Survived'], titanic['Embarked'], normalize = 'columns')

In [ ]:
pd.crosstab(titanic['Survived'], titanic['Embarked'], normalize = 'index')

In [ ]:
pd.crosstab(titanic['Survived'], titanic['Embarked'], normalize = 'all')

## 2.pivot + heatmap : 두 범주 집계 시각화

https://seaborn.pydata.org/generated/seaborn.heatmap.html

* 두 범주를 집계한 결과를 색의 농도로 표현해주는 그래프입니다.
* 집계(groupby)와 피봇(pivot)을 먼저 만들어 줘야 합니다.
* 여러 범주를 갖는 변수 비교 시 유용합니다.


In [ ]:
titanic.head()

In [ ]:
temp1 = titanic.groupby(['Embarked','Pclass'], as_index = False)['PassengerId'].count()
temp1.head()

In [ ]:
temp1.pivot('Embarked','Pclass', 'PassengerId')

In [ ]:
temp1 = titanic.groupby(['Embarked','Pclass'], as_index = False)['PassengerId'].count()
temp2 = temp1.pivot('Embarked','Pclass', 'PassengerId')
print(temp2)

sns.heatmap(temp2, annot = True)
plt.show()

In [ ]:
# 값을 정수로, 구간 간격을 살짝 벌려서 그리자.
sns.heatmap(temp2, annot = True, fmt = 'd', linewidth = .2)
plt.show()

* 추가 : pivot
    * pivot 함수를 이용하여 집계된 데이터를 재구성할 수 있습니다.
    * 문법 : dataframe.pivot(index, colums, values)


<img src='https://github.com/DA4BAM/image/blob/main/pivot.png?raw=true' width=600/>

### 연도별, 월별 평균 주가

① [groupby] stock 데이터를 이용하여 year별, month별 평균 주가(close)를 집계하시오.

In [ ]:
temp1 = stock.groupby(['Year','Month'], as_index = False)['Close'].mean()

② [pivot]  'Year','Month', 'Close' 으로 피봇시키시오.

In [ ]:
temp2 = temp1.pivot( 'Year','Month', 'Close')
temp2

③ heatmap을 그리시오.

In [ ]:
plt.figure(figsize = (15,6))
sns.heatmap(temp2, annot = True, fmt = '.1f', linewidth = .2)
plt.show()

## 3.시계열 데이터

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/retail_demand2.csv'
data = pd.read_csv(path, usecols = ['date',	'comp_sales'])
data = data.loc[data['date']<= '2015-12-31'].reset_index(drop = True)
data.columns = ['date',	'sales']
data.head(10)

In [ ]:
data['date'] = pd.to_datetime(data['date'])

### 1) 날짜 요소 뽑기

| 메서드 | 내용|
|----|----|
data['date'].dt.date         		| YYYY-MM-DD(문자)
data['date'].dt.year         		| 연(4자리숫자)
data['date'].dt.month        		| 월(숫자)
data['date'].dt.month_name()		| 월(문자)
data['date'].dt.day          		| 일(숫자)
data['date'].dt.time         		| HH:MM:SS(문자)
data['date'].dt.hour         		| 시(숫자)
data['date'].dt.minute       		| 분(숫자)
data['date'].dt.second       		| 초(숫자)
data['date'].dt.quarter       		| 분기(숫자)
data['date'].dt.day_name()  	| 요일이름(문자)
data['date'].dt.weekday       		| 요일숫자(0-월, 1-화) (=dayofweek)
data['date'].dt.weekofyear    		| 연 기준 몇주째(숫자) (=week)
data['date'].dt.dayofyear     		| 연 기준 몇일째(숫자)
data['date'].dt.days_in_month 	| 월 일수(숫자) (=daysinmonth)



* 요일

In [ ]:
data['date'].dt.day_name()

In [ ]:
# 문자열 길이를 첫 3글짜로 잘라서 저장하기
# .str.slice(시작, 끝)
data['date'].dt.day_name().str.slice(0, 3)

In [ ]:
data['weekday'] = data['date'].dt.day_name().str.slice(0, 3)
data.head()

### 2) shift

In [ ]:
temp = data[['date', 'sales']].copy()
temp.head()

In [ ]:
# shift(1)
temp['lag1'] = temp['sales'].shift()

# shift(2)
temp['lag2'] = temp['sales'].shift(2)

# shift(-1)
temp['lag_1'] = temp['sales'].shift(-1)
temp.head()

* 7일 전 판매량

In [ ]:
data['lag7'] = data['y'].shift(7)
data.head(10)

### 3) diff
특정 시점 대비 증감

In [ ]:
temp = data[['date', 'sales']].copy()
temp.head()

In [ ]:
# 전일대비 증감
temp['Diff1'] = temp['sales'].diff()

# 2일 전 대비 증가
temp['Diff2'] = temp['sales'].diff(2)

temp.head()

* 전일 대비 증감

In [ ]:
data['Diff1'] = data['sales'].diff()

* 7일 대비 증가

In [ ]:
data['Diff7'] = data['sales'].diff(7)

data.head(10)